In [1]:
import numpy as np
import cv2
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
def get_img_contour_thresh(img):
    x, y, w, h = 0, 0, 300, 300
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (35, 35), 0)
    ret, thresh1 = cv2.threshold(blur, 70, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    thresh1 = thresh1[y:y + h, x:x + w]
    contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    return img, contours, thresh1

In [3]:
! ls 

Digit-Recognizer             project_notes
README.md                    requirements.txt
data                         update_one.docx
dig_rec_scratch.ipynb        update_one.pdf
digit-recognizer-copy        update_one.txt
final-report-dl-2018.pdf     webcam_capture.py
logistic_regression_torch.py


In [24]:
import logistic_regression_torch as lr_torch

SyntaxError: invalid syntax (logistic_regression_torch.py, line 48)

In [3]:
path = Path('data/')

In [4]:
mnist = datasets.MNIST(path, download=True)

In [5]:
mnist

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: data
    Transforms (if any): None
    Target Transforms (if any): None

In [6]:
type(mnist)

torchvision.datasets.mnist.MNIST

In [7]:
batch_size = 64
test_batch_size = 1 # why are these different?

In [8]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py

train_dl = torch.utils.data.DataLoader(
    datasets.MNIST(path, train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_dl = torch.utils.data.DataLoader(
    datasets.MNIST(path, train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=False)

In [12]:
len(datasets.MNIST(path, train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])))

10000

In [50]:
# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/logistic_regression/main.py
# https://www.kaggle.com/negation/pytorch-logistic-regression-tutorial

class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__() # why are we supering ourselves?
        self.linear = nn.Linear(input_size, num_classes)


    def forward(self, x):
        # not sure it matters which one we use
        out = F.log_softmax(self.linear(x), dim = 0) # is this what I want? or wrap it in F.log_softmax()? 
        #out = self.linear(x)
        return out

input_dim = 28
input_size = input_dim**2
n_classes = 10
n_epochs = 10
learning_rate = 0.01

# Creating the model object
model = LogisticRegression(input_size, n_classes)

# Initiating loss and optimizer
# do we want SGD or Adam? read up on differences
# want something else for loss? CEL computes softmax internally

def train_epochs(model, train_dl = train_dl, n_epochs = 10, lr = 0.01, wd = 0.0):
    parameters = filter(lambda p: p.requires_grad, model.parameters()) # is this necessary?
    optimizer = optim.SGD(parameters, lr = lr)
    model.train()
    for epoch in np.arange(n_epochs):
        for i, (x, y) in enumerate(train_dl):
            x = x.reshape(-1, input_size)
            out = model(x)
            loss = F.cross_entropy(out, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if epoch % 2 == 0:
            print('Epoch ' + str(epoch) + ' train loss: ' + str(loss.item()))
    preds_list = test_loss(model, test_dl)
    #return preds_list


def test_loss(model, test_dl = test_dl):
    model.eval()
    total = 0
    correct = 0
    sum_loss = 0
    preds_list = list()

    for i, (x, y) in enumerate(test_dl):
        x = x.reshape(-1, input_size)
        batch = y.shape[0]
        out = model(x)
        loss = F.cross_entropy(out, y)
        preds = torch.argmax(out, dim = 1)
        sum_loss += batch*(loss.item())
        correct += preds.eq(y.data).sum().item()
        total += batch
        preds_list.append(preds)
        
    print("val loss and accuracy", sum_loss/total, correct/total)
    return preds_list


In [38]:
pred = train_epochs(model, n_epochs = 6)

Epoch 0 train loss: 0.3535199761390686
Epoch 2 train loss: 0.2776540517807007
Epoch 4 train loss: 0.7574548125267029
val loss and accuracy 0.2807054978674026 0.9193


In [39]:
pred

[tensor([ 7]),
 tensor([ 2]),
 tensor([ 1]),
 tensor([ 0]),
 tensor([ 4]),
 tensor([ 1]),
 tensor([ 4]),
 tensor([ 9]),
 tensor([ 6]),
 tensor([ 9]),
 tensor([ 0]),
 tensor([ 6]),
 tensor([ 9]),
 tensor([ 0]),
 tensor([ 1]),
 tensor([ 5]),
 tensor([ 9]),
 tensor([ 7]),
 tensor([ 3]),
 tensor([ 4]),
 tensor([ 9]),
 tensor([ 6]),
 tensor([ 6]),
 tensor([ 5]),
 tensor([ 4]),
 tensor([ 0]),
 tensor([ 7]),
 tensor([ 4]),
 tensor([ 0]),
 tensor([ 1]),
 tensor([ 3]),
 tensor([ 1]),
 tensor([ 3]),
 tensor([ 6]),
 tensor([ 7]),
 tensor([ 2]),
 tensor([ 7]),
 tensor([ 1]),
 tensor([ 2]),
 tensor([ 1]),
 tensor([ 1]),
 tensor([ 7]),
 tensor([ 4]),
 tensor([ 2]),
 tensor([ 3]),
 tensor([ 5]),
 tensor([ 1]),
 tensor([ 2]),
 tensor([ 4]),
 tensor([ 4]),
 tensor([ 6]),
 tensor([ 3]),
 tensor([ 5]),
 tensor([ 5]),
 tensor([ 6]),
 tensor([ 0]),
 tensor([ 4]),
 tensor([ 1]),
 tensor([ 9]),
 tensor([ 5]),
 tensor([ 7]),
 tensor([ 8]),
 tensor([ 9]),
 tensor([ 2]),
 tensor([ 7]),
 tensor([ 4]),
 tensor([ 

In [51]:
train_epochs(model, n_epochs = 6)

Epoch 0 train loss: 0.5242573022842407
Epoch 2 train loss: 0.7498753070831299
Epoch 4 train loss: 0.5521610379219055
val loss and accuracy 2.3025851249694824 0.098


In [36]:
% timeit train_epochs(model, n_epochs = 6)

Epoch 0 train loss: 0.3007313311100006
Epoch 2 train loss: 0.15727250277996063
Epoch 4 train loss: 0.38610872626304626
val loss and accuracy 0.28090187432396235 0.9218
Epoch 0 train loss: 0.43293774127960205
Epoch 2 train loss: 0.5466340184211731
Epoch 4 train loss: 0.520926296710968
val loss and accuracy 0.27457937731423954 0.9213
Epoch 0 train loss: 0.7348889112472534
Epoch 2 train loss: 0.2178499400615692
Epoch 4 train loss: 0.22468511760234833
val loss and accuracy 0.2702927966704033 0.9229
Epoch 0 train loss: 0.3722960948944092
Epoch 2 train loss: 0.12407708913087845
Epoch 4 train loss: 0.22541482746601105
val loss and accuracy 0.26783080963763345 0.9245
Epoch 0 train loss: 0.2675818204879761
Epoch 2 train loss: 0.15622666478157043
Epoch 4 train loss: 0.16603343188762665
val loss and accuracy 0.2701697267295805 0.9242
Epoch 0 train loss: 0.2707245647907257
Epoch 2 train loss: 0.13280612230300903
Epoch 4 train loss: 0.1264330893754959
val loss and accuracy 0.2681223748866436 0.9241

In [28]:
x, y = next(iter(train_dl))

x = x.reshape(-1, 28**2)

y

tensor([ 2,  5,  2,  1,  9,  0,  8,  8,  2,  9,  6,  7,  3,  6,
         2,  2,  7,  8,  7,  8,  9,  1,  5,  6,  4,  2,  8,  3,
         6,  2,  4,  1,  8,  6,  8,  2,  5,  2,  2,  5,  0,  3,
         3,  4,  7,  0,  9,  6,  4,  5,  6,  4,  9,  6,  9,  6,
         5,  4,  8,  3,  8,  5,  8,  3])

In [53]:
out = model(x)

F.cross_entropy(out, y)

tensor(0.2552)

In [24]:
out[:5]

tensor([[-2.1848, -2.1924, -1.7718, -2.4461, -2.9077, -2.2452, -2.0659,
         -2.4034, -2.9292, -2.4289],
        [-2.5110, -2.1021, -2.0598, -2.5032, -2.4736, -2.2438, -1.9029,
         -1.7479, -3.4522, -3.1694],
        [-2.3107, -3.8980, -3.6304, -2.7597, -2.5032, -1.7266, -1.2833,
         -1.8383, -2.5463, -4.1088],
        [-2.0000, -4.0957, -2.7099, -2.1744, -2.9172, -1.4791, -2.5378,
         -1.6593, -2.3165, -4.0215],
        [-2.7228, -3.8063, -2.2876, -2.5365, -2.9741, -1.7515, -1.7823,
         -1.9175, -1.8948, -3.1882]])

In [25]:
out.shape, y.shape

(torch.Size([64, 10]), torch.Size([64]))

In [59]:
torch.argmax(out, dim=1)

tensor([ 8,  5,  2,  1,  9,  0,  9,  8,  2,  9,  6,  7,  3,  6,
         2,  2,  7,  8,  7,  8,  9,  1,  5,  6,  4,  2,  8,  3,
         6,  2,  4,  1,  8,  6,  8,  2,  3,  2,  4,  5,  0,  3,
         3,  4,  7,  0,  9,  6,  4,  5,  6,  4,  7,  6,  9,  6,
         5,  4,  8,  3,  8,  5,  8,  3])

In [210]:
torch.argmax(out, dim=1)

tensor([ 1,  1,  0,  4,  4,  2,  2,  7,  9,  9,  1,  6,  3,  2,
         8,  9,  4,  4,  2,  8,  7,  8,  9,  1,  6,  1,  8,  5,
         1,  0,  6,  9,  3,  1,  6,  8,  0,  7,  6,  1,  9,  4,
         8,  5,  9,  6,  3,  3,  4,  3,  3,  3,  4,  8,  3,  8,
         5,  6,  6,  9,  5,  1,  7,  1])

In [43]:
output = model(x[0])

In [48]:
torch.argmax(output, dim = 0)

tensor(8)

In [52]:
output = model(x[0])
torch.argmax(output, dim = 0)

tensor(8)